In [1]:
import h5py
import torch
from selected_system import mols, mol_file
from ani_benchmark import NeighborBenchmark, FreeNeighborBenchmark
import pandas
import os
import tqdm
from IPython.display import display
tqdm.monitor_interval = 0

def pretty_print(orig_dict):
    """Pretty print times in a dictionary"""
    ret = {}
    for i in orig_dict:
        if orig_dict[i] is None:
            ret[i] = None
        else:
            ms = int(orig_dict[i])
            s = ms / 1000
            m = s / 60
            h = m / 60
            if ms < 1000:
                p = '{}ms'.format(ms)
            elif s < 60:
                p = '{:.2f}s'.format(s)
            elif m < 60:
                p = '{:.2f}min'.format(m)
            else:
                p = '{:.2f}min'.format(h)
            ret[i] = p
    return ret

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
torch.set_num_threads(1)
fm = h5py.File(os.path.join('../',mol_file), "r")

benchmarks = {
    'N,C': NeighborBenchmark(device=torch.device("cpu")),
    'F,C': FreeNeighborBenchmark(device=torch.device("cpu")),
}

if torch.cuda.is_available():
    benchmarks.update({
        'N,G': NeighborBenchmark(device=torch.device("cuda")),
        'F,G': FreeNeighborBenchmark(device=torch.device("cuda")),
    })

In [3]:
for i in mols:
    print('number of atoms:', i)
    smiles = mols[i]
    for s in smiles:
        print('Running benchmark on molecule', s)
        key = s.replace('/', '_')
        coordinates = torch.from_numpy(fm[key][()])
        species = fm[key].attrs['species'].split()
        results = {}
        for b in tqdm.tqdm_notebook(benchmarks):
            bench = benchmarks[b]
            coordinates = coordinates.type(bench.aev_computer.dtype)
            try:
                result = bench.oneByOne(coordinates, species)
            except RuntimeError as e:
                print(e)
                result = {'aev': None, 'energy': None, 'force': None}
            results[b + ',1'] = pretty_print(result)
            try:
                result = bench.inBatch(coordinates, species)
            except RuntimeError as e:
                print(e)
                result = {'aev': None, 'energy': None, 'force': None}
            results[b + ',B'] = pretty_print(result)
        df = pandas.DataFrame(results)
        display(df)
        break

number of atoms: 20
Running benchmark on molecule COC(=O)c1ccc([N+](=O)[O-])cc1


Widget Javascript not detected.  It may not be installed or enabled properly.


/usr/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


,"F,C,1","F,C,B","F,G,1","F,G,B","N,C,1","N,C,B","N,G,1","N,G,B"
aev,45.50s,2.52s,3.02min,279ms,46.43s,3.02s,3.03min,377ms
energy,2.56s,100ms,8.87s,8ms,2.57s,107ms,8.81s,13ms
force,54.26s,8.04s,3.28min,346ms,54.31s,7.93s,3.21min,329ms


number of atoms: 50
Running benchmark on molecule O=[N+]([O-])c1ccc(NN=Cc2ccc(C=NNc3ccc([N+](=O)[O-])cc3[N+](=O)[O-])cc2)c([N+](=O)[O-])c1


...

Widget Javascript not detected.  It may not be installed or enabled properly.


,"F,C,1","F,C,B","F,G,1","F,G,B","N,C,1","N,C,B","N,G,1","N,G,B"
aev,5.24s,57ms,18.23s,17ms,6.10s,61ms,18.59s,17ms
energy,987ms,22ms,2.42s,2ms,981ms,24ms,2.38s,2ms
force,7.14s,165ms,20.63s,21ms,7.03s,161ms,20.07s,20ms


number of atoms: 100
Running benchmark on molecule CC(C)C[C@@H](C(=O)O)NC(=O)C[C@@H]([C@H](CC1CCCCC1)NC(=O)CC[C@@H]([C@H](Cc2ccccc2)NC(=O)OC(C)(C)C)O)O


Widget Javascript not detected.  It may not be installed or enabled properly.


KeyboardInterrupt: 